In [94]:
#TODO: create functions for vapor line, feed line, and liquid line
#TODO: change the stage plotting to a single array (try appending values) (completed)
#TODO: rename functions from ref_calc to refCalc
#TODO: implement OOP to umbrella over functions
#TODO: ask for y* at the beginning
#TODO: create switch function to calculate the feed point based on inputs (liquid, vapor, mixed) (ask the user)


In [95]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_dark'
from IPython.display import clear_output

In [163]:
'''Specified Functions'''

def refCalc(a, b):
    '''
    DESCRIPTION
    Determines the reference temperature T_ref in degrees (C)
    
    INPUTS:
    a                  : Boiling point temperature of b
    b                  : Boiling point temperature of b

    OUTPUTS:
    T_ref              : Reference temperature (C) 
    
    '''
    if a < b:
        res = a
    else:
        res = b
    return res

def liquidEnthalpy(C, T_boil, mw):
    '''
    Description
    Returns the liquid enthalpy
    
    INPUTS:
    C                  : Average liquid specific heat
    T_boil             : Boiling point temperature
    mw                 : Molecular weight

    OUTPUTS:
    H                  : Liquid enthalpy 

    '''
    H = C*(T_boil - T_ref)*mw
    return H

def vaporEnthalpy(H_l,H_vap, mw):
    '''
    Description
    Returns the vapor enthalpy
    
    INPUTS:
    H_l                : Liquid enthalpy
    H_vap             : Latent heat of vaporization
    mw                 : Molecular weight

    OUTPUTS:
    H                  : Vapor enthalpy

    '''
    H = H_l + H_vap*mw
    return H

def vaporLineEq(i):
    '''
    DESCRIPTION
    Produces the vapor enthalpy across a range or specifed of compostiions

    INPUTS:
    i                 : Composition

    OUTPUTS:
    y                 : Output enthalpy(s)

    '''
    y = vaporLineSlope*i + Hb_v
    return y 

def liquidLineEq(i):
    '''
    DESCRIPTION
    Produces the liquid enthalpy across a range or specifed of compostiions

    INPUTS:
    i                 : Composition

    OUTPUTS:
    y                 : Output enthalpy(s)

    '''
    y = liquidLineSlope*i + Hb_l
    return y

def feedLineSlope():
    '''
    DESCRIPTION
    Calculates the feed lines slope to determine ideal stages.

    INPUTS:

    OUTPUTS:
    m                 : Feed line slope
    b                 : Feed line y intercept

    '''
    feedPoint = liquidLineSlope*xf + Hb_l
    feedPointThroughVapor = vaporLineSlope*yf_star + Hb_v
    m = (feedPointThroughVapor-feedPoint)/(yf_star-xf)
    b = -m*xf+feedPoint
    return m, b


def feedLineEq(i):
    '''
    DESCRIPTION
    Produces the feed enthalpy across a range or specifed of compostiions

    INPUTS:
    i                 : Composition

    OUTPUTS:
    y                 : Output enthalpy(s)
    '''
    
    y = feedLineSlope()[0]*i + feedLineSlope()[1]
    return y

'''Parameters'''
F = 100                             # Feed flow rate (kmol/hr)
xf=0.5                              # Feed composition
xd=0.96                             # Distillate composition
xb=0.04                             # Bottoms composition
feedType = 'saturated'              # Feed type

'''Properites:'''
mw_A=153.84                         # Molecular weight of a (g/mol)
mw_B=92.14                          # Molecular weight of b (g/mol)
Ta_boil=76.4                        # Boiling point of a (C)
Tb_boil=110.4                       # Boiling point of b (C)
T_ref = refCalc(Ta_boil,Tb_boil)   # Determined reference temperature
Ca=.225                             # Average liquid specific heat of a
Cb=.5                               # Average liquid specific heat of b
Ha_vap=46.42                        # Latent heat vap. of a @ 76.4
Hb_vap=86.8                         # Latent heat vap. of b @ 110.4

# enthalpy calculations
Ha_l = liquidEnthalpy(Ca, Ta_boil, mw_A)
Hb_l = liquidEnthalpy(Cb, Tb_boil, mw_B)
Ha_v = vaporEnthalpy(Ha_l, Ha_vap, mw_A)
Hb_v = vaporEnthalpy(Hb_l, Hb_vap, mw_B)

vaporLineSlope = (Ha_v-Hb_v)/1
liquidLineSlope = (Ha_l-Hb_l)/1

# intake feed vapor composition based on equilibrium data
while True:
    yf_star= float(input('Enter the feed vapor composition: ')) # 0.71 for now must be greater than xf
    if yf_star >= xf:
        break
    else:
        print('Enter a feed vapor composition greater than xf: {}'.format(xf))
    





# create a variable to iterate over compositions
x = np.linspace(0, 1, 101)

# make plot
fig1 = make_subplots(rows=1, cols=1)
fig1.add_scatter(x=x,                            
                 y=vaporLineEq(x), name='Vapor Line')
fig1.add_scatter(x=x,                            
                 y=liquidLineEq(x), name='Liquid Line')
fig1.add_scatter(x=x[int(xf*100):int(xd*100)+1],
                 y=feedLineEq(x[int(xf*100):int(xd*100)+1]), name='Feed Line')

fig1.add_shape(type='line', x0=xd, y0=-feedLineEq(xd), x1=xd, y1=feedLineEq(xd))
fig1.add_shape(type='line', x0=xb, y0=-feedLineEq(xb), x1=xb, y1=feedLineEq(xb))

fig1.update_layout(
    title='H vs comp',
    xaxis = dict(
        title = 'Composition (x_i,y_i)',
    ),
    yaxis = dict(
        title = ' Enthalpy (H)',
    )
    )

Enter a feed vapor composition greater than xf 0.5.
Enter a feed vapor composition greater than xf 0.5.
Enter a feed vapor composition greater than xf 0.5.


### Get functions sorted before editing below

In [154]:
xIdealStages = np.array([xd])
yIdealStages = np.array([vaporLineSlope*xIdealStages[0] + Hb_v])

stageCount = 0.0

fig2 = fig1

while xIdealStages[-1] > xb:
    x = np.array([0.0, 0.0, 0.0])
    y = np.array([0.0, 0.0, 0.0])

    x[0] = xIdealStages[-1]
    y[0] = yIdealStages[-1]

    x[1] = float(input('Enter the liquid composition: '))
    clear_output()
    y[1] = liquidLineSlope*x[1] + Hb_l
    x[2] = x[1]
    y[2] = vaporLineSlope*x[2] + Hb_v

    xIdealStages = np.append(xIdealStages, x[1:])
    yIdealStages = np.append(yIdealStages, y[1:])

    fig2.add_scatter(x=xIdealStages, y=yIdealStages)
    fig2.show()

    stageCount+=1
    
clear_output()

# calculate stages
idealStages = stageCount - (xb-xIdealStages[-1]/(xIdealStages[-3]-xIdealStages[-1]))

print('Ideal stages: ' + str(idealStages))

fig1.add_scatter(x=xIdealStages, y=yIdealStages, name='Ideal Stages')
fig1

ValueError: could not convert string to float: ''

In [ ]:
'''
(B) find the Minimum reflux ratio

MinRefluxRatio = (P' - V_1)\(V_1 - D)

V_1 is the vapor enthalpy at the first stage
D is the liquid enthalpy of the liquid distillate
P' is found graphically
'''
def P_prime(m, x, b):
    y = m*x +b
    return y

def R_min():
    P_prime = feedLineSlope*xd + b
    V_1 = vaporLineSlope*xd + Hb_v
    D = liquidLineSlope*xd + Hb_l

    return (P_prime-V_1)/(V_1-D)

R_min()

4.235561254207672

In [ ]:
'''
(C) find the ideal number of stages
Actual reflux ratio = Rmin * (multiplier)
Actual reflux ratio = [(H_D + Q_c/D) - V_1]/[V_1 - H_D]
Solve for '(H_D + Q_c/D)' or (P')
'''
multiplier = 2.05
actualRefluxRatio = minimumRefluxRatio * multiplier

P_prime = actualRefluxRatio*(V_1 - D) + V_1 # new point on xd for tie line
B_prime = -P_prime                          # new point on xb for tie line
P_prime

25711.35834895238

In [ ]:
feedLineSlope = (P_prime - B_prime)/(xd - xb)
feedLineSlope
feedLine = feedLineSlope*x + B_prime

def FeedLine(x):
    y = feedLineSlope*x + B_prime
    return y

In [ ]:
# create non-ideal plot
fig3 = make_subplots(rows=1, cols=1)
fig3.add_scatter(x=x, y=vaporLine, name='Vapor Line')
fig3.add_scatter(x=x, y=liquidLine, name='Liquid Line')
fig3.add_scatter(x=x, y=feedLine, name='Feed Line')
# fig3.add_scatter(x=[xd, xd], y=[-1000,feedLine[int(xd*100)]], name='xd')
# fig3.add_scatter(x=[xb, xb], y=[-1000,vaporLine[int(xb*100)]+5000], name='xb')
fig3.add_shape(type='line', x0=xd, y0=-FeedLine(xd), x1=xd, y1=FeedLine(xd))
fig3.add_shape(type='line', x0=xb, y0=-FeedLine(xb), x1=xb, y1=FeedLine(xb))

fig3.add_scatter(x=xIdealStages[0:2], y=yIdealStages[0:2], name='Ideal Stages')